In [13]:
import pandas as pd
import numpy as np

In [2]:
df_embs = pd.read_csv('data/embeddings.csv')

In [3]:
df_embs.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,prem
0,0.182002,0.912977,0.907421,-0.437929,0.538189,0.442572,-0.224152,0.259399,-0.527029,-0.540318,...,0.253885,-0.924071,-0.861477,0.871546,0.142212,-0.204056,0.294285,-0.269421,0.101756,dt_m1_connsp_2
1,-0.540258,0.849664,-0.699069,-0.810142,-0.198372,0.693475,0.483488,-0.086279,-0.615749,0.875644,...,-0.790588,-0.152291,0.631529,-1.045542,0.674132,0.585891,0.927966,-0.296319,0.750174,t7_connsp_2
2,-0.641890,0.398222,0.914349,0.315104,-0.860476,0.767970,-0.695319,-0.999168,0.638714,-0.150963,...,-0.221978,0.275682,0.659623,0.584907,0.467428,-0.707116,-0.289522,0.144035,1.043148,t66_tmap_1
3,0.103218,-0.838002,-0.185052,-0.640600,-0.997660,-0.846374,0.961225,-0.072874,-0.158189,0.070172,...,0.209781,-0.346835,-0.427865,-0.762210,-0.008766,-0.482838,0.670356,-0.823398,0.771485,t85_tmap_1
4,-0.470709,0.841319,1.005236,0.683271,0.520221,-0.657198,-0.402885,-0.967556,0.872271,0.341585,...,-0.036094,-0.757505,-0.638248,0.462436,0.366449,-0.589190,0.219308,0.131029,0.548225,t87_tmap_1


In [5]:
indexes_train = []
premises = []

ones = []

with open('homework2_exercise2/data/dependencies_train', 'r') as f:
    for line in f.readlines():
        index, *prem = line.split(':')
        prem = ''.join(prem).split()
        
        indexes_train.append(index)
        premises.append(prem)
        
        for p in prem:
            ones.append((index, p))

In [6]:
ones_df = pd.DataFrame(ones, columns=['theorem', 'premise']).drop_duplicates()

In [7]:
ones_df.head()

,theorem,premise
0,t3_xboole_0,d1_xboole_0
1,t3_xboole_0,fc1_xboole_0
2,t3_xboole_0,d4_xboole_0
3,t3_xboole_0,d7_xboole_0
4,t7_xboole_0,d1_xboole_0


In [10]:
chronology = pd.read_csv('homework2_exercise2/data/chronology', squeeze=True, names=['chronology'])
def get_available_theorems(name):
    return chronology.loc[:(chronology == name).argmax()-1]

In [11]:
used_premises = ones_df.groupby('theorem')['premise'].unique()

In [14]:
FALSE_EXAMPLES_RATIO = 3

dataset_zeros = []
for theorem, theorem_premises in ones_df.groupby('theorem')['premise']:
    available_theorems = get_available_theorems(theorem)
    zero_premises = np.setdiff1d(available_theorems, theorem_premises)
    
    zeros_chosen = zero_premises[np.random.randint(0, len(zero_premises), FALSE_EXAMPLES_RATIO * theorem_premises.shape[0])]
    dataset_zeros += list(zip(
        [theorem] * len(zeros_chosen),
        zeros_chosen
    ))

/home/piotrek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
zeros_df = pd.DataFrame(dataset_zeros, columns=['theorem', 'premise'])

In [46]:
zeros_df['useful'] = 0
ones_df['useful'] = 1
df_merged = pd.concat([zeros_df, ones_df])

In [18]:
df_merged['useful'].value_counts()

0    82299
1    27433
Name: useful, dtype: int64

In [29]:
df_merged.head()

,theorem,premise,useful
0,t100_tmap_1,fc12_struct_0,0
1,t100_tmap_1,t103_tops_1,0
2,t100_tmap_1,t61_setfam_1,0
3,t100_tmap_1,t52_yellow_0,0
4,t100_tmap_1,t138_funct_1,0


In [20]:
df_embs.columns[-1]

'prem'

In [32]:
df_feats = df_embs

In [94]:
def map_features(thm_prem_pairs):
    df_values = pd.merge(
        thm_prem_pairs, df_feats,
        left_on=['theorem'],
        right_on=['prem']
    )
    df_values = pd.merge(
        df_values, df_feats,
        left_on=['premise'],
        right_on=['prem'],
        suffixes=('_thm', '_prem')
    ).drop(['prem_thm', 'prem_prem'], axis=1)
    return df_values

In [48]:
df_merged = map_features(df_merged)

## Training

In [55]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [56]:
model.fit(df_merged.drop(['theorem', 'premise', 'useful'], axis=1), df_merged['useful'])

/home/piotrek/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [57]:
y_pred = model.predict_proba(df_merged.drop(['theorem', 'premise', 'useful'], axis=1))

In [59]:
y_pred

array([[0.86594307, 0.13405693],
       [0.89496822, 0.10503178],
       [0.89175673, 0.10824327],
       ...,
       [0.76068205, 0.23931795],
       [0.8146189 , 0.1853811 ],
       [0.61769084, 0.38230916]])

In [60]:
from sklearn.metrics import roc_auc_score
roc_auc_score(df_merged['useful'], y_pred[:, 1])

0.7717245664570043

In [64]:
import lightgbm as lgb

In [109]:
# train test split
from sklearn.model_selection import train_test_split

df_train = df_merged

train_theorems, test_theorems = train_test_split(df_train['theorem'].unique(), shuffle=True, train_size=0.75)

train = df_train['theorem'].isin(train_theorems)
test = df_train['theorem'].isin(test_theorems)

features = list(df_train.columns.drop(['theorem', 'premise', 'useful']))
y_name = 'useful'

In [110]:
train_data = lgb.Dataset(df_train[train][features], df_train[train][y_name],
                        #categorical_feature=features,
                        free_raw_data=False
                        )
test_data = lgb.Dataset(df_train[test][features], df_train[test][y_name],
                        #categorical_feature=features,
                        free_raw_data=False
                        )

In [111]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 14,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 20,
    'learning_rate': 0.05,
 #   'lambda_l2': 0.03,
    'verbose': 1,
    'threads': 6,
    'num_boost_round': 400
}
model = lgb.train(
    parameters,
    train_data
)

In [130]:
model = LogisticRegression()
model.fit(df_train[train][features], df_train[train][y_name])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

## Finding Proofs

In [113]:
def get_premises(conj):
    premises = get_available_theorems(conj)
    premises.name = 'premise'
    premises = premises.to_frame()
    premises['theorem'] = conj
    df_values = map_features(premises)
    
    df_X = df_values.drop(['theorem', 'premise'], axis=1)
    #premises['useful'] = (model.predict(df_X) + model_small.predict(df_X)) / 2
    df_values['useful'] = model.predict(df_X)
    premises = df_values.sort_values(by='useful', ascending=False)
    return premises['premise']

In [114]:
import sys
sys.path.append('..')
from homework2_exercise2.run_E_prover import find_proof

In [115]:
statements_path = 'homework2_exercise2/data/statements'
dirpath = 'homework2_exercise2/new_conjectures/'
with open(statements_path, 'r') as f:
    statements_lines = f.read().splitlines()
statements = {}
for l in statements_lines:
    statements[l.split(',')[0].replace('fof(', '')] = l

In [116]:
import warnings
warnings.filterwarnings('ignore')

In [117]:
test_theorems.shape

(284,)

In [132]:
conj_results = []
proved_cnt = 0
for i, conj_name in enumerate(test_theorems):
    
    premises_for_conj = get_premises(conj_name)
    for num_prems in map(lambda x: 2**x, range(10)):
        proved = find_proof(conj_name, premises_for_conj[:num_prems], statements, dirpath)
        if proved:
            break
    if i % 10 == 0:
        print('=' * 20)
        print(f'i={i}, proved {proved_cnt}')
    
    if proved:
        proved_cnt += 1
    conj_results.append((conj_name, proved))

Proof of conjecture t150_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t150_tmap_1.E_output 
Proof of conjecture t150_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t150_tmap_1.E_output 
Proof of conjecture t150_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t150_tmap_1.E_output 
Proof of conjecture t150_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t150_tmap_1.E_output 
Proof of conjecture t150_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t150_tmap_1.E_output 
Proof of conjecture t150_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t150_tmap_1.E_output 
Proof of conjecture t150_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t150_tmap_1.E_output 
Proof of conjecture t150_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t150_tmap_1.E_output 
Proof of conjecture t150

Proof of conjecture t46_enumset1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t46_enumset1.E_output 
Proof of conjecture t73_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t73_funct_2.E_output 
Proof of conjecture t73_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t73_funct_2.E_output 
Proof of conjecture t73_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t73_funct_2.E_output 
Proof of conjecture t73_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t73_funct_2.E_output 
Proof of conjecture t73_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t73_funct_2.E_output 
Proof of conjecture t73_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t73_funct_2.E_output 
Proof of conjecture t73_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t73_funct_2.E_output 
Proof of conjecture t7

Proof of conjecture t106_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t106_tmap_1.E_output 
Proof of conjecture t106_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t106_tmap_1.E_output 
Proof of conjecture t106_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t106_tmap_1.E_output 
Proof of conjecture t106_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t106_tmap_1.E_output 
Proof of conjecture t24_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t24_tmap_1.E_output 
Proof of conjecture t24_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t24_tmap_1.E_output 
Proof of conjecture t24_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t24_tmap_1.E_output 
Proof of conjecture t24_tmap_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t24_tmap_1.E_output 
Proof of conjecture t24_tmap_1 N

Proof of conjecture t72_tops_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t72_tops_1.E_output 
i=20, proved 3
Proof of conjecture t48_setfam_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_setfam_1.E_output 
Proof of conjecture t48_setfam_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_setfam_1.E_output 
Proof of conjecture t48_setfam_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_setfam_1.E_output 
Proof of conjecture t48_setfam_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_setfam_1.E_output 
Proof of conjecture t48_setfam_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_setfam_1.E_output 
Proof of conjecture t48_setfam_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_setfam_1.E_output 
Proof of conjecture t48_setfam_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_setfam_1.E_outpu

Proof of conjecture t5_wellord1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t5_wellord1.E_output 
Proof of conjecture t5_wellord1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t5_wellord1.E_output 
Proof of conjecture t5_wellord1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t5_wellord1.E_output 
Proof of conjecture t5_wellord1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t5_wellord1.E_output 
Proof of conjecture t60_tex_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t60_tex_2.E_output 
Proof of conjecture t60_tex_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t60_tex_2.E_output 
Proof of conjecture t60_tex_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t60_tex_2.E_output 
Proof of conjecture t60_tex_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t60_tex_2.E_output 
Proof of conjecture t60_tex_2 NOT found.

...

Proof of conjecture t48_funct_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_funct_1.E_output 
Proof of conjecture t48_funct_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_funct_1.E_output 
Proof of conjecture t48_funct_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_funct_1.E_output 
Proof of conjecture t48_funct_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_funct_1.E_output 
Proof of conjecture t48_funct_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_funct_1.E_output 
Proof of conjecture t48_funct_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t48_funct_1.E_output 
Proof of conjecture t111_enumset1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t111_enumset1.E_output 
Proof of conjecture t111_enumset1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t111_enumset1.E_output 
Proof of conject

Proof of conjecture t115_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t115_funct_2.E_output 
Proof of conjecture t115_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t115_funct_2.E_output 
Proof of conjecture t115_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t115_funct_2.E_output 
Proof of conjecture t115_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t115_funct_2.E_output 
Proof of conjecture t115_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t115_funct_2.E_output 
Proof of conjecture t115_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t115_funct_2.E_output 
Proof of conjecture t30_filter_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t30_filter_1.E_output 
Proof of conjecture t30_filter_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t30_filter_1.E_output 
Proof of

Proof of conjecture t35_tex_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t35_tex_2.E_output 
Proof of conjecture t35_tex_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t35_tex_2.E_output 
Proof of conjecture t35_tex_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t35_tex_2.E_output 
Proof of conjecture t35_tex_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t35_tex_2.E_output 
Proof of conjecture t27_finset_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t27_finset_1.E_output 
Proof of conjecture t27_finset_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t27_finset_1.E_output 
Proof of conjecture t27_finset_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t27_finset_1.E_output 
Proof of conjecture t27_finset_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t27_finset_1.E_output 
Proof of conjecture t27_finset_1

Proof of conjecture t32_tops_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t32_tops_1.E_output 
Proof of conjecture t32_tops_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t32_tops_1.E_output 
Proof of conjecture t32_tops_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t32_tops_1.E_output 
Proof of conjecture t32_tops_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t32_tops_1.E_output 
Proof of conjecture t32_tops_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t32_tops_1.E_output 
i=270, proved 63
Proof of conjecture t152_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t152_funct_2.E_output 
Proof of conjecture t152_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t152_funct_2.E_output 
Proof of conjecture t152_funct_2 NOT found. Output saved in file homework2_exercise2/new_conjectures/t152_funct_2.E_output 
Proof of co

Proof of conjecture t23_xboole_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t23_xboole_1.E_output 
Proof of conjecture t23_xboole_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t23_xboole_1.E_output 
Proof of conjecture t23_xboole_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t23_xboole_1.E_output 
Proof of conjecture t23_xboole_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t23_xboole_1.E_output 
Proof of conjecture t23_xboole_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t23_xboole_1.E_output 
Proof of conjecture t23_xboole_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t23_xboole_1.E_output 
Proof of conjecture t49_relset_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t49_relset_1.E_output 
Proof of conjecture t49_relset_1 NOT found. Output saved in file homework2_exercise2/new_conjectures/t49_relset_1.E_output 
Proof of

In [133]:
res_logreg = pd.DataFrame(conj_results)
res_logreg.to_csv('homework2_exercise2/logreg.csv', index=False)

In [134]:
res_logreg[1].mean()

0.2323943661971831

In [129]:
res_rand[1].mean()

0.23591549295774647

In [121]:
res[1].mean()

0.2711267605633803

In [125]:
premises_for_conj.sample(frac=1)

961                 t30_ordinal1
1868                t16_lattice3
367                 t92_zfmisc_1
2035                rc1_yellow_1
1059    redefinition_k3_relset_1
15                   d1_xboole_0
1794              dt_m2_filter_1
1127                 t29_mcart_1
795                   d4_funct_1
1666                  cc4_tops_1
159                t107_xboole_1
873                  t86_funct_1
827                  t40_funct_1
981                  d2_wellord1
1182                 t74_mcart_1
1189                 t82_mcart_1
17       idempotence_k3_xboole_0
1026                 d7_wellord1
523                 t11_setfam_1
2014                 t4_yellow_1
1464                t65_pre_topc
1212                 t9_wellord2
575                  fc1_relat_1
275                  t1_zfmisc_1
905                 t157_funct_1
991                  t9_wellord1
1675                   d7_tops_1
1375               fc10_finset_1
960                 t29_ordinal1
473                 t42_subset_1
          